In [3]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append("/home/lcz/lenlp/text-generation/py_project")

## 配置GPU使用

In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

## 加载数据集

In [5]:
from utils.data_loader import load_dataset, load_test_dataset
from utils.linux_config import embedding_matrix_path
from utils.wv_loader import load_embedding_matrix, load_vocab

root path: /home/lcz/lenlp/text-generation


In [6]:
train_X, train_Y, test_X = load_dataset()

In [7]:
embedding_matrix = load_embedding_matrix("./../data/embedding_matrix")

In [8]:
vocab_path = "./../data/vocab.txt"
vocab, reverse_vocab = load_vocab(vocab_path)

## 配置参数

In [9]:
units = 1024
params = {}
params["vocab_size"] = len(vocab)
params["embed_size"] = 300
params["enc_units"] = units
params["attn_units"] = units
params["dec_units"] = units
params["batch_size"] = 64
params["epochs"] = 2
params["max_enc_len"] = 200
params["max_dec_len"] = 41